In [ ]:
import numpy as np  
import pandas as pd  
import rasterio  
import os  
import matplotlib.pyplot as plt  
import seaborn as sns  
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler  

plt.rcParams['font.family'] = 'Arial'  
plt.rcParams['font.sans-serif'] = ['Arial']  # Specify default font  
plt.rcParams['axes.unicode_minus'] = False

# Set data file path  
data_path = r'D:\Rnmap\CLip\S'  

# Raster file mapping  
raster_files = {  
    'Ra': 'ra.tif',  
    'CEC': 'cec.tif',  
    'Clay': 'clay.tif',  
    'DTB': 'dtb.tif',  
    'OC': 'oc.tif',  
    'PH': 'ph.tif',  
    'Lithology': 'glim.tif',
    'Sand': 'sand.tif',
    'Silt': 'silt.tif'
}  

# Read raster image and convert to pandas Series  
def read_raster(filename, data_path):  
    file_path = os.path.join(data_path, filename)  
    with rasterio.open(file_path) as f:  
        data = f.read(1)  
        nodata_value = f.nodata  
    
    # Handle NoData values  
    processed_data = data.astype(float)  
    processed_data[data == nodata_value] = np.nan  
    
    return pd.Series(processed_data.flatten(), name=filename.split('.')[0].upper())  

# Read all rasters using dictionary comprehension  
processed_data = {name: read_raster(file, data_path)   
                  for name, file in raster_files.items()}  

# Convert to DataFrame  
df = pd.DataFrame(processed_data)  

# Remove rows containing NaN  
df_cleaned = df.dropna()  

# Feature names  
feature_names = [ 'CEC', 'Clay', 'DTB', 'OC', 'PH', 'Lithology', 'Sand', 'Silt', 'Ra']  

# Data distribution visualization  
plt.figure(figsize=(16, 8))  

# Feature distribution - kernel density estimation  
plt.subplot(1, 2, 1)  
for feature in feature_names[:-1]:  # Exclude Ra  
    sns.kdeplot(df_cleaned[feature], fill=True, label=feature)  
plt.title("Feature Distribution", fontsize=16)  
plt.xlabel("Feature", fontsize=14)  
plt.ylabel("Density", fontsize=14)  
plt.legend()  

# Feature correlation heatmap  
plt.subplot(1, 2, 2)  
corr = df_cleaned.corr()  
sns.heatmap(corr,   
            cmap='coolwarm',   
            center=0,  
            annot=True,  
            fmt=".2f",   
            square=True,   
            linewidths=.5,   
            cbar_kws={"shrink": .8})  
plt.title("Feature Correlation Heatmap", fontsize=18)  
plt.tight_layout()  
plt.show()  

# Print features most correlated with Ra  
ra_correlations = corr['Ra'].sort_values(key=abs, ascending=False)  
print("\nFeatures most correlated with Ra:")  
print(ra_correlations)  

# Split features and target variable  
X = df_cleaned[feature_names[:-1]]  # Exclude Ra  
y = df_cleaned['Ra']  

# Split into training and testing sets  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)  

# Data standardization  
scaler = StandardScaler()  
X_scaled = pd.DataFrame(  
    scaler.fit_transform(X_train),   
    columns=X_train.columns  
)  

# Print data information  
print("Training set features shape:", X_train.shape)  
print("Testing set features shape:", X_test.shape)  
print("Training set target variable shape:", y_train.shape)  
print("Testing set target variable shape:", y_test.shape)
    